In [ ]:
#Library loading
library(UpSetR)
library(dplyr)
library(ggplot2)
library(tibble)
library(phyloseq)
library(reshape2)
library(scales)
library(paletteer)
library(hrbrthemes)
library(ecodist)
library(gplots)
library(RColorBrewer)
library(ape)
library(vegan)
library(gridExtra)
library(ggrepel)
library(forcats)
library(microbiome)
library(Biobase)
library(tidyr)
library(agricolae)
library(dendextend)
library(svglite)

In [ ]:
#REFERENCES PANGENOME#

In [ ]:
#Obtaining UpSet plot
pang_c=read.table("pangenome_references.tsv", row.names=1)
pang_c=pang_c[,-c(1)]
pang_m=replace(pang_c, pang_c > 0, 1)
pangenome_ups <- upset(pang_m, nsets = 36, number.angles = 0, point.size = 1.5, line.size = 0.5, text.scale=4,  
      mainbar.y.label = "Protein intersections", sets.x.label = "Proteins per sample", nintersects=NA, 
                        order.by="freq")
pangenome_ups
pdf("pang_ref_upset_complete.pdf", height=20, width=70)
pangenome_ups
dev.off()

In [ ]:
#Using the annotation from COG to generate functional profile heatmap#
Solirubrobacter_c = read.table("pang_eggnog_counts.tsv", row.names=1)
colnames(Solirubrobacter_c) = c("COG_Category","PangenomeRef","CoreRef","Sginsenosidimutans","Spauli",
                                "SpCPCC","Sphytolaccae","SpURHD","Ssoli","Staibaiensis")
Solirubrobacter_c=Solirubrobacter_c[,-1]
head(Solirubrobacter_c)

In [ ]:
#Standardizing per row using Z values
z_ref_c <- t(Solirubrobacter_c)
z_ref_c <- scale(z_ref_c,center = TRUE, scale = TRUE)
Solirubrobacter_c=t(Solirubrobacter_c)
rc_w.bray=distance(Solirubrobacter_c, "bray-curtis")
rc_w.clust=hclust(rc_w.bray)
plot(rc_w.clust, hang=-1)
#Creating heatmap
paleta <- colorRampPalette(brewer.pal(9, "PuOr"))
z_ref_c=t(z_ref_c)
dim(z_ref_c)
heatmap.2(z_ref_c, trace="none", Colv=as.dendrogram(rc_w.clust), col=paleta)

svg("hm2_eggnog_ref_2ways.svg", height=17, width=10)
heatmap.2(z_ref_c, trace="none", Colv=as.dendrogram(rc_w.clust), col=paleta)
dev.off()

In [ ]:
#Creating bar plot with CDS per sample of the references#
cds = t(read.table("cds_strains.tsv", row.names=1)) #This table was generated from each reference genome information
rownames(cds) = c("CDS")
svg("cds_ref.svg")
cds_plot=barplot(cds, las="2")
cds_plot
dev.off()

In [ ]:
#Gene families analysis#
gene_c = read.table("all.genefam.counts.tsv", row.names=1, header=TRUE)
colnames(gene_c) = c("Cluster","Soil_1","Soil_2","Soil_3","Soil_4","Rhizosphere_1","Rhizosphere_2","Rhizosphere_3","Rhizosphere_4","Rhizosphere_5","Soil_5","Soil_6","Rhizosphere_6","Rhizosphere_7","Rhizosphere_8","Rhizosphere_9","Rhizosphere_10","Sginsenosidimutans","Spauli","SpCPCC","Sphytolaccae","SpURHD","Ssoli","Staibaiensis")
gene_c=gene_c[,2:24]
max(gene_c)

In [ ]:
#For only singeltons
gene_1=replace(gene_c, gene_c > 1, 0)
#Removing rows having all zeros
gene_1=gene_1[rowSums(gene_1[])>0,]
write.table(gene_1,"gene_1.tsv", quote=FALSE, sep='\t')
dim(gene_1)
max(gene_1)
#For families with 2-5 genes
gene_25=replace(gene_c, gene_c < 2, 0)
gene_25=replace(gene_25, gene_25 > 5, 0)
#Removing rows having all zeros
gene_25=gene_25[rowSums(gene_25[])>0,]
write.table(gene_25,"gene_25.tsv", quote=FALSE, sep='\t')
dim(gene_25)
For families with more than 6 genes
gene_6=replace(gene_c, gene_c < 6, 0)
#Removing rows having all zeros
gene_6=gene_6[rowSums(gene_6[])>0,]
write.table(gene_6,"gene_6.tsv", quote=FALSE, sep='\t')
dim(gene_6)
#Obtaining families that belong to the same representative sequence (a total of 8 for families of 6-9 genes)
gene_6_d=replace(gene_6, gene_6>0, 1)
write.table(gene_6_d,"gene_6_d.tsv", quote=FALSE, sep='\t')

In [ ]:
t=read.table("gene_fam_6_ref_2.tsv", header=TRUE, row.names = 1)
dim(t)
dataframe <- data.frame(t)

In [ ]:
#Creating bubble plot for gene familes with six or more genes
ggplot(dataframe, aes(x = Species, y = Protein,
                      size = Size,
                      color = COG_Category))+
geom_point(alpha = 0.8)+ theme_light()+
scale_size(range = c(5, 22), name = "Number of GF")+
  scale_colour_paletteer_d(("ggsci::default_igv")) 

ggsave("gene_fam6_bubbles.svg", height=9, width=10)

In [ ]:
#ENVIRONMENTAL EXPANDED PANGENOME (EEP)#

In [ ]:
#Constructing functional profile using COG annotation#
#Constructing dendrogram
pang_a = data.matrix(read.table("pang_new_eggnog_counts.tsv", row.names=1, header=TRUE))
pang_a=t(pang_a[,-c(1)])
pa.bray=distance(pang_a, "bray-curtis")
pa.clust=hclust(pa.bray)
plot(pa.clust, hang=-1)
#Standardizing per row using Z value
pang_a = data.matrix(read.table("pang_new_eggnog_counts.tsv", row.names=1, header=TRUE))
pang_a=t(pang_a[,-c(1)])
z_pang_a <- scale(pang_a,center = TRUE, scale = TRUE)
#Plotting heatmap
paleta <- colorRampPalette(brewer.pal(9, "PuOr"))
z_pang_a=t(z_pang_a)
heatmap.2(z_pang_a, trace="none", Colv=as.dendrogram(pa.clust), col=paleta)

In [ ]:
#EEP diversity analysis

In [ ]:
#Loading count table
p=as.matrix(read.table("pang_new_eggnog_counts2.tsv", header = TRUE))
colnames(p)=c("Soil_1","Soil_2","Soil_3","Soil_4","Rhizosphere_1","Rhizosphere_2","Rhizosphere_3","Rhizosphere_4","Rhizosphere_5","Sginsenosidimutans","Soil_5","Soil_6","Spauli","SpCPCC","Sphytolaccae","SpURHD","Ssoli","Staibaiensis","Rhizosphere_6","Rhizosphere_7","Rhizosphere_8","Rhizosphere_9","Rhizosphere_10","PangenomeExt","CoreRef","PangenomeRef")
PROT = otu_table(p, taxa_are_rows=T)
#Loading annotation table
tax=as.matrix(read.table("allcounts_tax.tsv",row.names=1,fill=TRUE))
TAXA=tax_table(tax)
colnames(TAXA)=c("COG_category")
#Creating phyloseq object
prot2=phyloseq(PROT,TAXA)
#Loading metadata table
prot_data=read.table("allcounts_data.tsv", header=TRUE, row.names=1, sep="\t")
sampledata=sample_data(data.frame(samples=prot_data$samples,country=prot_data$country,soil_type=prot_data$soil_type,origin=prot_data$origin,host=prot_data$host,type=prot_data$type,dummy=prot_data$dummy,cluster=prot_data$Cluster,comp=prot_data$comp,row.names=sample_names(prot2)))
#Creating phyloseq object
prot=phyloseq(PROT,TAXA,sampledata)
prot

In [ ]:
#Using ANOSIM to determine significance 
tPROT=t(PROT)
prot_CAP.bray=vegdist(decostand(tPROT,"normalize"),method="bray")
ans_c = anosim(tPROT, sampledata$country, distance = "bray", permutations = 9999) 
ans_c 
ans_o = anosim(tPROT, sampledata$origin, distance = "bray", permutations = 9999)
ans_o 
ans_h = anosim(tPROT, sampledata$host, distance = "bray", permutations = 9999)
ans_h 
ans_s = anosim(tPROT, sampledata$soil_type, distance = "bray", permutations = 9999)
ans_s 
ans_2 = anosim(tPROT, sampledata$comp, distance = "bray", permutations = 9999)
ans_2
ans_cl = anosim(tPROT, sampledata$cluster, distance = "bray", permutations = 9999) 
ans_cl

In [ ]:
#Plotting PCoA ordination using Bray-curtis dissimilarity matrix
prot.nmds.bray=ordinate(prot, "PCoA", "bray")
plot_ordination(prot, prot.nmds.bray, shape="type", type="samples",color="host",title="PCoA Bray")+
    geom_text(mapping = aes(label = samples), size = 3, vjust = -0.9, hjust=1) + theme_bw() + 
    geom_point(size = 5)+scale_colour_paletteer_d("ggsci::springfield_simpsons")
ggsave("pcoabray_all_annot.svg")